Noteneingabetool für HISinOne
=============================
Dieses Jupyter-Notebook ist ein interaktives Tool zur Noteneingabe in HISinOne. Es können exportierte HISInOne Exceltabellen geladen werden, woraus eine interaktive Tabelle zum Eingeben der Punkte in einzelnen Aufgaben erzeugt wird. Aus der Summe der Punkte können Noten berechnet werden sowie Statistken wie z.B. die Notenverteilung dargstellt werden.

Die Noten können dann wieder im HISinOne Format exportiert werden und in HISinOne hochgeladen werden.

---
Konfiguration:
--------------

In [ ]:
# Die zu importierende HISInOne-Tabelle (über die Funktion Excel-Export der Noteneingabe in HISInOne erzeugt)
table_name = '5160-RocketScience-WiSe_2023.xlsx' 
# Namen der Aufgaben, für die Punkte eintragen werden sollen
questions = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
# Parameter für Noteberechnung
max_points = 65     # Maximale Punktzahl
grade_1_0 = 0.92    # Prozentuale Grenze für Note 1,0
grade_4_0 = 0.4     # Prozentuale Grenze für Note 4,0

---
Namensimport
------------

In [ ]:
import pandas as pd
import numpy as np

st = pd.read_excel(table_name, header=None)

# Finde Indizes der Zeilen mit den Werten "startHISsheet" und "endHISsheet"
def find_row_with_value(df, value):
    mask = st.iloc[:, 0] == value
    if not mask.any():
        raise ValueError(f'{value} nicht gefunden')    
    return mask.idxmax()

start_idx = find_row_with_value(st, 'startHISsheet')
end_idx = find_row_with_value(st, 'endHISsheet')
#print(start_idx,end_idx)
# Lade Daten im korrekten Bereich
st = pd.read_excel(table_name,
                   header=start_idx+1,
                   nrows=end_idx-start_idx-2,
                   index_col='Matrikelnummer',
                   dtype={'Leistung': str})

print('Folgende Tabelle wurde importiert:')
st

---
Punkteeingabe
-------------

In [ ]:
# from pathlib import Path
import panel as pn
pn.extension('tabulator')
pn.widgets.Tabulator.theme = 'modern'
from bokeh.models.widgets.tables import NumberFormatter

# Erzeuge Tabelle zur Punkteeingabe
select_columns = st[['Nachname', 'Vorname']]
new_col = pd.Series([np.nan] * len(st), name='1', index=st.index)
st_points = pd.concat([select_columns, new_col], axis=1)
add_columns = [pd.Series([np.nan] * len(st), name=q, index=st.index) for q in questions]
st_points = pd.concat([select_columns] + add_columns, axis=1)

# Speichere Tabelle zur Punkteeingabe bzw. lade sie, falls sie bereits existiert
storage_path = Path() / Path(table_name + '_editable_df.pkl')

# ACHTUNG: Folgende Zeile auskommentieren zum Zurücksetzen der Änderungen! Dies ist irreversibel!
#storage_path.unlink()

if storage_path.exists():
    print(f'Stelle Änderungen aus {storage_path} wieder her.')
    df = pd.read_pickle(storage_path)
else:
    print(f'Keine Änderungen gefunden, werde zukünftige Änderungen nach {storage_path} speichern.')
    df = st_points


bokeh_formatters = {
    'Matrikelnummer': NumberFormatter(format='0')
}

header_filters = {
    'Matrikelnummer': {'type': 'input', 'placeholder': 'Filter'},
    'Nachname': {'type': 'input', 'placeholder': 'Filter'}
}

tabulator_editors = {
    'Matrikelnummer': None,
    'Nachname': None,
    'Vorname': None,
}

df_widget = pn.widgets.Tabulator(df,
                                 layout='fit_data',
                                 height=500,
                                 width=1000,
                                 formatters=bokeh_formatters,
                                 frozen_columns=['Matrikelnummer', 'Nachname'],
                                 header_filters=header_filters,
                                 editors=tabulator_editors)

df_widget.on_edit(lambda e: df.to_pickle(storage_path))

display(df_widget)


---
Notenschlüssel und Notenberechnung
-------------


In [ ]:
# Notenschlüssel
import numpy as np

grades = ['1,0', '1,3', '1,7', '2,0', '2,3', '2,7', '3,0', '3,3', '3,7', '4,0', '5,0']

grade_interval = (grade_1_0 - grade_4_0) / 9

grade_intervals = [grade_1_0 - i * grade_interval for i in range(10)]
grade_intervals.append(0)
points = np.array([i * max_points for i in grade_intervals])

score_table = pd.DataFrame({'Note': grades, 'Prozent': grade_intervals, 'Untergrenze': points})

print(f'Maximalpunktzahl: {max_points}')
print(f'1.0-Grenze (%): {grade_1_0}')
print(f'4.0-Grenze (%): {grade_4_0}')
print('-----')
print('Notentabelle:')
score_table

In [ ]:
# Berechne Summe und Note
df['Summe'] = df[questions].sum(axis=1, min_count=len(questions))

df['Note'] = df['Summe'].apply(lambda x: np.nan if np.isnan(x) else score_table.iloc[np.digitize(x, points)]['Note'])

print('Folgende Punktesummen und Noten wurden berechnet:')
df

---
Notenverteilung
---------------

In [ ]:
pd.options.plotting.backend = "plotly"

score_hist = pd.DataFrame({'Anzahl': [0] * len(grades)}, index=score_table['Note'])

counts = pd.DataFrame(df['Note'].value_counts().rename('Anzahl'))
score_hist.update(counts)

print('Notenverteilung:')
print(score_hist)

number_students = len(df)
number_participants = len(df[df['Summe'].notna()])
passed = 1-score_hist.iloc[-1]["Anzahl"]/number_participants
print(f'Anzahl Anmeldungen: {number_students}')
print(f'Anzahl Teilnehmer: {number_participants}')
print(f'Bestanden: {passed*100:.2f} %')
print(f'Durchgefallen: {100-passed*100:.2f} %')

score_hist.plot(kind='bar')

---
Export
---------------

In [ ]:
# Update Noten
st['Leistung'] = df['Note'].replace({np.nan: 'NT'})
st_out = st.reset_index()

# Examplan.id muss erste Spalte sein (es wird nicht auf den Namen geprüft)
st_out = st_out.reindex(columns=['Examplan.id'] + [col for col in st_out.columns if col != 'Examplan.id'])

# Lese Anfang der originalen Tabelle (for HisInOne Header)
export_table = pd.read_excel(table_name, nrows=start_idx+1, header=None)

# Konkateniere Header und aktualisierte Tabelle
export_table = pd.DataFrame(np.vstack([export_table.values,                                         # HISinOne Header
                                       np.array(list(st_out)),                                      # Tabellenheader (Matrikelnummer, Nachname, Vorname, ...)
                                       st_out.values,                                               # Daten
                                       np.array(['endHISsheet'] + [''] * (len(list(st_out))-1))     # HISinOne Footer
                                      ]), columns=export_table.columns)

# Exportiere Tabelle
export_table.to_excel(table_name + '_Noten.xlsx', sheet_name='First Sheet', header=False, index=False)